In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np
import torch

In [2]:
# Training dataset
df = pd.read_csv('train.csv')
df.head(8)

,'Imagen','Sex','Description'
0,'img1.png',1,'Male'
1,'img2.png',0,'Female'
2,'img3.png',1,'Male'
3,'img4.png',0,'Female'
4,'img5.png',0,'Female'
5,'img6.png',1,'Male'
6,'img7.png',0,'Female'
7,'img8.png',1,'Male'


In [3]:
# Read Sex Column
y_true = np.genfromtxt('train.csv', delimiter=',', 
                       skip_header=1, usecols = (1))
print(y_true)

[1. 0. 1. 0. 0. 1. 0. 1.]


In [4]:
# Keran One-Hot
num_classes = 2 # 2 classes Male and Female
keras_y_true = keras.utils.to_categorical(y_true, num_classes=num_classes)
print(keras_y_true)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [5]:
# Pytorch One-Hot
num_classes = 2 # 2 classes Male and Female
pytorch_y_true = torch.tensor( y_true, dtype = torch.int64) # numpy array to tensor
pytorch_y_true = torch.nn.functional.one_hot(pytorch_y_true, num_classes=num_classes) # one-hot funtion
print(pytorch_y_true)

tensor([[0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1]])


In [6]:
# Ground Truth (expected prediction)
y_true = [[0, 1], # Female
          [1, 0], # Male
          [0, 1], # Female
          [1, 0], # Male
          [1, 0], # Male
          [0, 1], # Female
          [1, 0], # Male
          [0, 1]] # Female

In [7]:
y_pred = [[-0.0181,  0.1989], # Female  (ok)    0.1989 higher than -0.0181
          [ 0.5594, -0.1696], # Male    (ok)
          [ 1.4101,  1.0020], # Male    (wrong)
          [-0.1596, -0.4885], # Male    (ok)    -0.1596 higher than -0.4885
          [-1.2368,  0.1567], # Female  (wrong)
          [-0.2297, -0.5287], # Male    (wrong)
          [ 1.4883, -0.6566], # Male    (ok)
          [ 0.0336,  0.2778]] # Female  (ok)

In [8]:
# Therefore we missed (wrong prediction) 3 out of 8
# 5/8 = 0.625 (this is the expected categorical accuracy output)
total = len(y_true)
missed = 3
(total-missed)/total

0.625

In [9]:
# Keras Categorical Accuracy
cat_acc = keras.metrics.CategoricalAccuracy()
cat_acc.update_state(y_pred, y_true)
print(cat_acc.result().numpy())


0.625


In [10]:
# Pytorch Categorical Accuracy
def pytorch_cat_acc(y_pred, y_true):
  aa, preds = torch.max(y_pred, dim=1)
  correct = (preds == torch.argmax(y_true, dim=1)).float()
  accuracy = correct.sum() / len(correct)
  return accuracy

In [11]:
pytorch_cat_acc(
  torch.tensor(y_pred, dtype=torch.float32), 
  torch.tensor(y_true, dtype=torch.float32)).item()

0.625